In [3]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from IPython.display import display

import config

In [4]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 2
TOTAL_BUDGET = 1000

In [5]:
current_team_names = {"André Onana",
                      "Matz Sels",
                      "Gabriel dos Santos Magalhães",
                      "Michael Keane",
                      "Wout Faes",
                      "Jack Stephens",
                      "Cristian Romero",
                      "Morgan Rogers",
                      "Bryan Mbeumo",
                      "Facundo Buonanotte",
                      "Luis Díaz",
                      "Mohamed Salah",
                      "Yoane Wissa",
                      "Erling Haaland",
                      "Chris Wood"
                      }

In [6]:
with open("data/data.csv","r",encoding="utf-8") as f:
    all_player_data = pd.read_csv(f)

In [7]:
current_team = Team.fromNameSet(current_team_names, all_player_data)

In [8]:
current_team_cost = current_team.getTotalCost()
current_team_cost

1002

In [9]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)
data = all_data["data"][SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame(data=data)

In [10]:
selected_team = Team.fromDataFrame(selected_team_df)

In [11]:
print(selected_team)


Total Score: 0

Goalkeepers:
- Jordan Pickford (Captain) 	Score: 0	Cost: 49
- Łukasz Fabiański (Vice Captain) 	Score: 0	Cost: 40
Defenders:
- Murillo Santiago Costa dos Santos	Score: 0	Cost: 45
- Joško Gvardiol	Score: 0	Cost: 63
- Ola Aina	Score: 0	Cost: 48
- Virgil van Dijk	Score: 0	Cost: 63
- Jan Bednarek	Score: 0	Cost: 40
Attackers:
- Chris Wood	Score: 0	Cost: 66
- Matheus Santos Carneiro Da Cunha	Score: 0	Cost: 68
- Danny Ings	Score: 0	Cost: 50
Midfielders:
- Cole Palmer	Score: 0	Cost: 109
- Bruno Borges Fernandes	Score: 0	Cost: 84
- Bryan Mbeumo	Score: 0	Cost: 79
- Mohamed Salah	Score: 0	Cost: 129
- Harry Winks	Score: 0	Cost: 45


In [12]:
new_players = selected_team - current_team
print(new_players)


Total Score: 0

Goalkeepers:
- Jordan Pickford (Captain) 	Score: 0	Cost: 49
- Łukasz Fabiański (Vice Captain) 	Score: 0	Cost: 40
Defenders:
- Murillo Santiago Costa dos Santos	Score: 0	Cost: 45
- Joško Gvardiol	Score: 0	Cost: 63
- Ola Aina	Score: 0	Cost: 48
- Virgil van Dijk	Score: 0	Cost: 63
- Jan Bednarek	Score: 0	Cost: 40
Attackers:
- Matheus Santos Carneiro Da Cunha	Score: 0	Cost: 68
- Danny Ings	Score: 0	Cost: 50
Midfielders:
- Cole Palmer	Score: 0	Cost: 109
- Bruno Borges Fernandes	Score: 0	Cost: 84
- Harry Winks	Score: 0	Cost: 45


In [13]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

978

In [14]:
added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

Added cost: -24


In [15]:
surplus = current_team_cost - new_team_cost

In [17]:
HEURISTIC = "total_points"
new_players.calculateScore(HEURISTIC)
selected_team.calculateScore(HEURISTIC)
current_team.calculateScore(HEURISTIC)

In [18]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 594.4875000000002


In [19]:
def too_expensive():
    pass

In [20]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    bestTransfer = max(allTransfers)
    if(bestTransfer.getScoreDif() > 0):
        return bestTransfer
    else:
        return None

In [21]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [22]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [23]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)

    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [24]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team):
    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    return newTeam

In [25]:
newTeam = getNewTeam(current_team, selected_team, new_players)
newTeam = newTeam.toBenchTeam()
print(newTeam)

Best transfer:
Transfer from Erling Haaland -> Matheus Santos Carneiro Da Cunha:
Old player: Erling Haaland	Score: 185.11	Cost: 152
New player: Matheus Santos Carneiro Da Cunha	Score: 318.6	Cost: 68
- Cost change: -84
- Score change: 133.48
# Starting 11:

Total Score: 1630.7849999999999

Goalkeepers:
- André Onana	Score: 90.1	Cost: 51
Defenders:
- Jack Stephens	Score: 23.62	Cost: 40
- Gabriel dos Santos Magalhães	Score: 19.12	Cost: 61
- Cristian Romero	Score: 18.78	Cost: 50
- Wout Faes	Score: 18.69	Cost: 42
Attackers:
- Matheus Santos Carneiro Da Cunha (Vice Captain) 	Score: 318.6	Cost: 68
- Yoane Wissa	Score: 168.08	Cost: 61
Midfielders:
- Mohamed Salah (Captain) 	Score: 559.07	Cost: 129
- Bryan Mbeumo	Score: 219.45	Cost: 79
- Morgan Rogers	Score: 104.06	Cost: 54
- Luis Díaz	Score: 91.2	Cost: 77

# Bench:

Total Score: 209.33999999999997

Goalkeepers:
- Matz Sels	Score: 48.37	Cost: 47
Defenders:
- Michael Keane	Score: 18.4	Cost: 43
Attackers:
- Chris Wood	Score: 111.69	Cost: 66
Midfi